<a href="https://colab.research.google.com/github/ashwin-yedte/visual-intelligence-travel-finance/blob/main/VL_Encoding_Framework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VL ENCODING FRAMEWORK** - WITH COMPREHENSIVE IMAGE PREPROCESSING
Visual-Language Encoding Infrastructure for Indian Travel Destinations

COMPLETE FEATURE SET:
1.  CLIP tensor extraction bug
2. Comprehensive image preprocessing (EXIF, RGB, aspect ratio)
3. Image validation and quality checks
4. Prompt validation (normalization, token limits)
5. Quality reports and statistics
6. Error recovery and logging

PREPROCESSING PIPELINES:
- IMAGE: EXIF orientation → RGB conversion → Aspect ratio → Padding → Validation
- PROMPT: Token validation → Duplicate detection → Quality metrics

OBJECTIVES:
1. Pre-compute CLIP embeddings for all landmark images
2. Extract semantic prompts using 2,200 CLIP prompt library
3. Create searchable database for similarity matching
4. Enable two-stage matching (visual + semantic)


WORKFLOW:
- Stage 1: Pre-compute prompt embeddings (one-time, 2,200 prompts)
- Stage 2: Process each image (extract CLIP embedding + prompts)
- Stage 3: Aggregate per destination
- Stage 4: Create search indices

================================================================

INSTALL PACKAGES

================================================================

In [34]:
!pip install -q transformers torch pillow tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("PACKAGES INSTALLED")
print("="*80)

PACKAGES INSTALLED


================================================================

IMPORT PACKAGES

================================================================

In [35]:
from google.colab import drive
import os
import json
import numpy as np
from PIL import Image, ImageOps
import torch
from transformers import CLIPModel, CLIPProcessor
from tqdm import tqdm
from datetime import datetime
import pickle
import warnings
from typing import Tuple, Dict, Any, List

warnings.filterwarnings('ignore')



# ================================================================
   SETUP AND MOUNT Google Drive   
# ================================================================


In [36]:
print("="*80)
print("VISUAL LANGUAGE ENCODING FRAMEWORK")
print("="*80)

drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/visual-intelligence-travel-finance'

LANDMARKS_PATH = f'{BASE_PATH}/data/landmarks'
METADATA_PATH = f'{LANDMARKS_PATH}/metadata.json'
PROMPT_LIBRARY_PATH = f'{BASE_PATH}/data/prompt_library/clip_prompts_india_themes_semantic.json'

VL_ENCODING_PATH = f'{BASE_PATH}/data/vl_encoding'
EMBEDDINGS_PATH = f'{VL_ENCODING_PATH}/embeddings'
PROMPTS_PATH = f'{VL_ENCODING_PATH}/prompts'
REPORTS_PATH = f'{VL_ENCODING_PATH}/reports'

os.makedirs(EMBEDDINGS_PATH, exist_ok=True)
os.makedirs(f'{PROMPTS_PATH}/image_prompts', exist_ok=True)
os.makedirs(REPORTS_PATH, exist_ok=True)

print(f"Directories created")
print("="*80)




VISUAL LANGUAGE ENCODING FRAMEWORK
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Directories created


================================================================

STEP 1: PROMPT VALIDATION

================================================================

    
    Validates and analyzes text prompts for CLIP processing.
    Features:
    - Token length validation (CLIP limit: 77 tokens)    
    - Quality metrics and reporting
    

In [37]:
print("\n" + "="*80)
print("PROMPT VALIDATION MODULE")
print("="*80)

class PromptValidator:
    def __init__(self, processor: CLIPProcessor):
        """
        Initialize validator with CLIP processor.

        Args:
            processor: CLIPProcessor for tokenization
        """
        self.processor = processor
        self.validation_log = []
        print("PromptValidator initialized")

    def validate_prompt(self, prompt: str, prompt_id: str = None) -> Dict[str, Any]:
        """
        Validate a single prompt.

        Args:
            prompt: Text prompt to validate
            prompt_id: Optional identifier for logging

        Returns:
            Dictionary with validation results
        """
        issues = []
        warnings = []

        if not prompt or not prompt.strip():
            issues.append("Empty prompt")
            return {
                'valid': False,
                'issues': issues,
                'warnings': warnings,
                'prompt': prompt,
                'prompt_id': prompt_id
            }

        prompt_clean = prompt.strip()

        char_length = len(prompt_clean)
        if char_length < 5:
            warnings.append(f"Very short prompt ({char_length} chars)")
        if char_length > 150:
            warnings.append(f"Long prompt ({char_length} chars)")

        try:
            tokens = self.processor.tokenizer(prompt_clean, truncation=False)
            token_count = len(tokens['input_ids'])

            if token_count > 77:
                issues.append(f"Exceeds CLIP token limit: {token_count}/77 tokens")
            elif token_count > 60:
                warnings.append(f"Near token limit: {token_count}/77 tokens")

        except Exception as e:
            issues.append(f"Tokenization failed: {str(e)}")
            token_count = -1

        problem_chars = ['@', '#', '$', '%', '^', '*', '|', '\\']
        found_chars = [c for c in problem_chars if c in prompt_clean]
        if found_chars:
            warnings.append(f"Contains special characters: {found_chars}")

        result = {
            'valid': len(issues) == 0,
            'issues': issues,
            'warnings': warnings,
            'prompt': prompt_clean,
            'prompt_id': prompt_id,
            'char_length': char_length,
            'token_count': token_count if token_count != -1 else None
        }

        self.validation_log.append(result)
        return result

    def validate_prompt_library(self, prompt_library: Dict) -> Dict[str, Any]:
        """
        Validate entire prompt library.

        Args:
            prompt_library: Dictionary of themes -> categories -> prompts

        Returns:
            Validation report with statistics
        """
        print("Validating prompt library...")

        report = {
            'total_prompts': 0,
            'valid_prompts': 0,
            'prompts_with_issues': 0,
            'prompts_with_warnings': 0,
            'issues_found': [],
            'warnings_found': [],
            'token_stats': {
                'min': float('inf'),
                'max': 0,
                'mean': 0,
                'over_limit': 0
            }
        }

        token_counts = []

        for theme, categories in prompt_library.items():
            for category, prompts in categories.items():
                for idx, prompt in enumerate(prompts):
                    prompt_id = f"{theme}/{category}/{idx}"

                    result = self.validate_prompt(prompt, prompt_id)
                    report['total_prompts'] += 1

                    if result['valid']:
                        report['valid_prompts'] += 1
                    else:
                        report['prompts_with_issues'] += 1
                        report['issues_found'].extend([
                            {'prompt_id': prompt_id, 'issue': issue}
                            for issue in result['issues']
                        ])

                    if result['warnings']:
                        report['prompts_with_warnings'] += 1
                        report['warnings_found'].extend([
                            {'prompt_id': prompt_id, 'warning': warning}
                            for warning in result['warnings']
                        ])

                    if result['token_count'] is not None:
                        token_counts.append(result['token_count'])
                        if result['token_count'] > 77:
                            report['token_stats']['over_limit'] += 1

        if token_counts:
            report['token_stats']['min'] = int(min(token_counts))
            report['token_stats']['max'] = int(max(token_counts))
            report['token_stats']['mean'] = float(np.mean(token_counts))

        return report

    def generate_report(self, validation_results: Dict, output_file: str = None) -> str:
        """
        Generate human-readable validation report.

        Args:
            validation_results: Results from validate_prompt_library()
            output_file: Optional path to save report

        Returns:
            Report text
        """
        report_lines = []
        report_lines.append("=" * 80)
        report_lines.append("PROMPT LIBRARY VALIDATION REPORT")
        report_lines.append("=" * 80)
        report_lines.append("")

        report_lines.append("SUMMARY")
        report_lines.append("-" * 80)
        report_lines.append(f"Total prompts: {validation_results['total_prompts']}")
        report_lines.append(f"Valid prompts: {validation_results['valid_prompts']}")
        report_lines.append(f"Prompts with issues: {validation_results['prompts_with_issues']}")
        report_lines.append(f"Prompts with warnings: {validation_results['prompts_with_warnings']}")
        report_lines.append("")

        report_lines.append("TOKEN STATISTICS")
        report_lines.append("-" * 80)
        stats = validation_results['token_stats']
        report_lines.append(f"Min tokens: {stats['min']}")
        report_lines.append(f"Max tokens: {stats['max']}")
        report_lines.append(f"Mean tokens: {stats['mean']:.1f}")
        report_lines.append(f"Prompts over limit (77): {stats['over_limit']}")
        report_lines.append("")

        if validation_results['issues_found']:
            report_lines.append("CRITICAL ISSUES")
            report_lines.append("-" * 80)
            for item in validation_results['issues_found']:
                report_lines.append(f"  [{item['prompt_id']}] {item['issue']}")
            report_lines.append("")
        else:
            report_lines.append("No critical issues found")
            report_lines.append("")

        if validation_results['warnings_found']:
            report_lines.append("WARNINGS")
            report_lines.append("-" * 80)
            for item in validation_results['warnings_found'][:10]:
                report_lines.append(f"  [{item['prompt_id']}] {item['warning']}")
            if len(validation_results['warnings_found']) > 10:
                report_lines.append(f"  ... and {len(validation_results['warnings_found']) - 10} more")
            report_lines.append("")

        report_lines.append("=" * 80)

        report_text = "\n".join(report_lines)

        if output_file:
            with open(output_file, 'w') as f:
                f.write(report_text)
            print(f"Report saved to: {output_file}")

        return report_text


print("PromptValidator class ready")
print("="*80)


PROMPT VALIDATION MODULE
PromptValidator class ready


================================================================

STEP 2: IMAGE PREPROCESSING

================================================================

    Comprehensive image preprocessing for CLIP model.
    
    Handles:
    - EXIF orientation correction
    - RGB conversion from any color mode
    - Aspect ratio preservation
    - High-quality resampling
    - White padding for consistent dimensions


In [38]:
print("\n" + "="*80)
print("IMAGE PREPROCESSING")
print("="*80)

class ImagePreprocessor:
    """Comprehensive image preprocessing for CLIP."""

    def __init__(self, target_size: Tuple[int, int] = (224, 224)):
        self.target_size = target_size
        print(f"ImagePreprocessor initialized (target: {target_size})")

    def preprocess_image(self, image_path: str) -> Image.Image:
        """Load and preprocess image."""
        try:
            img = Image.open(image_path)
            img = self._fix_orientation(img)

            if img.mode != 'RGB':
                if img.mode == 'RGBA':
                    background = Image.new('RGB', img.size, (255, 255, 255))
                    background.paste(img, mask=img.split()[3])
                    img = background
                else:
                    img = img.convert('RGB')

            img = self._resize_with_padding(img, self.target_size)
            return img

        except Exception as e:
            raise Exception(f"Failed to preprocess {image_path}: {str(e)}")

    def _fix_orientation(self, img: Image.Image) -> Image.Image:
        """Fix EXIF orientation."""
        try:
            img = ImageOps.exif_transpose(img)
        except:
            pass
        return img

    def _resize_with_padding(self, img: Image.Image, target_size: Tuple[int, int]) -> Image.Image:
        """Resize with aspect ratio preservation."""
        img_ratio = img.width / img.height
        target_ratio = target_size[0] / target_size[1]

        if img_ratio > target_ratio:
            new_width = target_size[0]
            new_height = int(new_width / img_ratio)
        else:
            new_height = target_size[1]
            new_width = int(new_height * img_ratio)

        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)

        canvas = Image.new('RGB', target_size, (255, 255, 255))
        offset_x = (target_size[0] - new_width) // 2
        offset_y = (target_size[1] - new_height) // 2
        canvas.paste(img, (offset_x, offset_y))

        return canvas

    def validate_image(self, image_path: str, max_size_mb: float = 10.0) -> Dict[str, Any]:
        """Validate image."""
        try:
            if not os.path.exists(image_path):
                return {'valid': False, 'error': 'File not found'}

            size_mb = os.path.getsize(image_path) / (1024 * 1024)
            if size_mb > max_size_mb:
                return {'valid': False, 'error': f'File too large: {size_mb:.2f}MB'}

            img = Image.open(image_path)
            img_format = img.format
            dimensions = img.size
            img.close()

            return {'valid': True, 'size_mb': size_mb, 'format': img_format, 'dimensions': dimensions}

        except Exception as e:
            return {'valid': False, 'error': str(e)}


preprocessor = ImagePreprocessor(target_size=(224, 224))
print("="*80)


IMAGE PREPROCESSING
ImagePreprocessor initialized (target: (224, 224))


================================================================

STEP 3: LOAD CLIP MODEL

================================================================

In [39]:
print("\n" + "="*80)
print("LOADING CLIP MODEL")
print("="*80)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

model_name = "openai/clip-vit-base-patch32"
print(f"Model: {model_name}")
print("Loading...")

model = CLIPModel.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

model.to(device)
model.eval()

sample_text = ["test"]
inputs = processor(text=sample_text, return_tensors="pt", padding=True)
inputs = {k: v.to(device) for k, v in inputs.items()}
with torch.no_grad():
    test_output = model.get_text_features(**inputs)
    if torch.is_tensor(test_output):
        embedding_dim = test_output.shape[-1]
    else:
        embedding_dim = 512

print(f"\nModel loaded")
print(f"  Parameters: {sum(p.numel() for p in model.parameters())/1e6:.1f}M")
print(f"  Embedding dimension: {embedding_dim}")
if device == "cuda":
    print(f"  GPU: {torch.cuda.get_device_name(0)}")

print("="*80)


LOADING CLIP MODEL
Device: cpu
Model: openai/clip-vit-base-patch32
Loading...


Loading weights:   0%|          | 0/398 [00:00<?, ?it/s]

CLIPModel LOAD REPORT from: openai/clip-vit-base-patch32
Key                                  | Status     |  | 
-------------------------------------+------------+--+-
text_model.embeddings.position_ids   | UNEXPECTED |  | 
vision_model.embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Model loaded
  Parameters: 151.3M
  Embedding dimension: 512


================================================================

STEP 4: LOAD IMAGE META DATA  AND PROMPT LIBRARY

================================================================

In [60]:
print("\n" + "="*80)
print("LOADING DATA")
print("="*80)

with open(METADATA_PATH, 'r') as f:
    metadata = json.load(f)

print(f"Metadata: {metadata['total_images']} images, {metadata['total_destinations']} destinations")

# ADD THIS: Initialize pipeline_status if it doesn't exist
if 'pipeline_status' not in metadata:
    metadata['pipeline_status'] = {
        'embeddings_computed': False,
        'prompts_extracted': False,
        'prompts_validated': False
    }
    print("Initialized pipeline_status in metadata")

with open(PROMPT_LIBRARY_PATH, 'r') as f:
    prompt_library = json.load(f)

total_prompts = sum(sum(len(prompts) for prompts in categories.values())
                    for categories in prompt_library.values())
print(f"Prompt library: {total_prompts} prompts")

print("="*80)


LOADING DATA
Metadata: 215 images, 47 destinations
Initialized pipeline_status in metadata
Prompt library: 2200 prompts


================================================================

STEP 5: VALIDATE PROMPT LIBRARY

================================================================

In [41]:
print("\n" + "="*80)
print("VALIDATING PROMPT LIBRARY")
print("="*80)

validator = PromptValidator(processor)
validation_results = validator.validate_prompt_library(prompt_library)

validation_report = validator.generate_report(
    validation_results,
    output_file=f'{REPORTS_PATH}/prompt_validation_report.txt'
)

print("\n" + validation_report)

if validation_results['prompts_with_issues'] > 0:
    print(f"\nWARNING: {validation_results['prompts_with_issues']} prompts have critical issues!")
    print("Review the validation report before proceeding.")
else:
    print("\nAll prompts passed validation!")

print("="*80)



VALIDATING PROMPT LIBRARY
PromptValidator initialized
Validating prompt library...
Report saved to: /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/reports/prompt_validation_report.txt

PROMPT LIBRARY VALIDATION REPORT

SUMMARY
--------------------------------------------------------------------------------
Total prompts: 2200
Valid prompts: 2200
Prompts with issues: 0
Prompts with warnings: 0

TOKEN STATISTICS
--------------------------------------------------------------------------------
Min tokens: 17
Max tokens: 28
Mean tokens: 21.3
Prompts over limit (77): 0

No critical issues found


All prompts passed validation!


================================================================

EMBEDDING EXTRACTION FUNCTIONS

================================================================

In [52]:
print("\n" + "="*80)
print("DEFINING EXTRACTION FUNCTIONS")
print("="*80)

def extract_clip_features(outputs):
    """Universal fix for extracting projected tensors from CLIP outputs."""
    # Prioritize 'image_embeds' or 'text_embeds' which are the *projected* CLIP features (512-dim for base-patch32)
    if hasattr(outputs, 'image_embeds') and torch.is_tensor(outputs.image_embeds):
        return outputs.image_embeds
    if hasattr(outputs, 'text_embeds') and torch.is_tensor(outputs.text_embeds):
        return outputs.text_embeds

    # If the output itself is already a tensor, return it directly.
    if torch.is_tensor(outputs):
        return outputs

    # Added: Check for 'pooler_output' which contains the 512-dim projected embedding for BaseModelOutputWithPooling
    if hasattr(outputs, 'pooler_output') and torch.is_tensor(outputs.pooler_output):
        return outputs.pooler_output

    # If still not found, raise an error for clarity.
    raise TypeError(f"Could not extract 512-dim projected CLIP features from model output type: {type(outputs)}. Object: {outputs}. "
                    f"Expected direct tensor or object with 'image_embeds'/'text_embeds'/'pooler_output' tensor attribute.")


def extract_image_embedding(image_path, model, processor, preprocessor, device):
    """Extract CLIP embedding from image with preprocessing."""
    try:
        validation = preprocessor.validate_image(image_path)
        if not validation['valid']:
            print(f"Skipping {image_path}: {validation['error']}")
            return None

        img = preprocessor.preprocess_image(image_path)

        inputs = processor(images=img, return_tensors="pt", padding=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.get_image_features(**inputs)
            image_features = extract_clip_features(outputs)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)

        return image_features.cpu().numpy()[0]

    except Exception as e:
        print(f"\nERROR in extract_image_embedding for {image_path}: {e}")
        return None


def extract_text_embeddings_batch(texts, model, processor, device):
    """Extract CLIP embeddings for batch of text prompts."""
    try:
        inputs = processor(text=texts, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.get_text_features(**inputs)
            text_features = extract_clip_features(outputs)
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)

        return text_features.cpu().numpy()

    except Exception as e:
        print(f"\nERROR in extract_text_embeddings_batch: {e}")
        return None


print("Extraction functions defined")
print("="*80)



DEFINING EXTRACTION FUNCTIONS
Extraction functions defined


================================================================

STEP 6: PRE-COMPUTE PROMPT EMBEDDINGS


    Pre-compute embeddings for all 2,200 prompts
    
    FIXED: Properly extract tensor from CLIP output object
    
    Returns:
        Dictionary with structure:
        {
            'Beach': {
                'LandscapeType': [
                    {'text': 'prompt text', 'embedding': np.array(512,)},
                    ...
                ],
                ...
            },
            ...
        }


================================================================

In [53]:
print("\n" + "="*80)
print("STAGE 6: PRE-COMPUTING PROMPT EMBEDDINGS")
print("="*80)

def precompute_prompt_embeddings(prompt_library, model, processor, device):
    """Pre-compute embeddings for all prompts."""

    prompt_embeddings = {}
    total_prompts = sum(sum(len(p) for p in cats.values()) for cats in prompt_library.values())

    print(f"Total prompts: {total_prompts}")

    with tqdm(total=total_prompts, desc="Computing prompt embeddings") as pbar:
        for theme, categories in prompt_library.items():
            prompt_embeddings[theme] = {}

            for category, prompts in categories.items():
                prompt_embeddings[theme][category] = []

                batch_size = 32
                for i in range(0, len(prompts), batch_size):
                    batch = prompts[i:i+batch_size]

                    embeddings = extract_text_embeddings_batch(batch, model, processor, device)

                    if embeddings is not None:
                        for j, prompt_text in enumerate(batch):
                            prompt_embeddings[theme][category].append({
                                'text': prompt_text,
                                'embedding': embeddings[j]
                            })

                    pbar.update(len(batch))

    return prompt_embeddings


prompt_embeddings_file = f'{EMBEDDINGS_PATH}/prompt_embeddings.pkl'

# Force re-computation to ensure consistency after fix
force_recompute_prompts = True

if os.path.exists(prompt_embeddings_file) and not force_recompute_prompts:
    print("Found cached prompt embeddings...")

    with open(prompt_embeddings_file, 'rb') as f:
        cached_embeddings = pickle.load(f)

    for theme, cats in cached_embeddings.items():
        for cat, prompts in cats.items():
            if prompts:
                cached_dim = prompts[0]['embedding'].shape[0]
                break
        break

    print(f"  Cached dimension: {cached_dim}")
    print(f"  Current model dimension: {embedding_dim}")

    if cached_dim == embedding_dim:
        print("  Dimensions match - using cache")
        prompt_embeddings = cached_embeddings
    else:
        print("  Dimension mismatch - recomputing...")
        os.remove(prompt_embeddings_file)
        prompt_embeddings = precompute_prompt_embeddings(prompt_library, model, processor, device)

        with open(prompt_embeddings_file, 'wb') as f:
            pickle.dump(prompt_embeddings, f)
        print(f"  Saved new embeddings")
else:
    if os.path.exists(prompt_embeddings_file) and force_recompute_prompts:
        print("Forcing recomputation of prompt embeddings, ignoring cached file.")
        os.remove(prompt_embeddings_file) # Ensure old cache is removed
    else:
        print("Computing prompt embeddings...")

    prompt_embeddings = precompute_prompt_embeddings(prompt_library, model, processor, device)

    with open(prompt_embeddings_file, 'wb') as f:
        pickle.dump(prompt_embeddings, f)
    print(f"Saved to: {prompt_embeddings_file}")

print("="*80)



STAGE 6: PRE-COMPUTING PROMPT EMBEDDINGS
Forcing recomputation of prompt embeddings, ignoring cached file.
Total prompts: 2200


Computing prompt embeddings:   1%|▏         | 32/2200 [00:00<01:01, 35.08it/s]


ERROR in extract_text_embeddings_batch: Could not extract 512-dim projected CLIP features from model output type: <class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>. Object: BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.0360,  0.1090, -0.1890,  ...,  1.4989,  1.6122, -1.4643],
         [ 1.2186, -1.3370, -0.2337,  ...,  1.6266,  1.2361,  0.0886],
         ...,
         [-0.4441, -1.5782,  0.3321,  ..., -0.8385, -0.6500,  0.6907],
         [-0.5024, -1.6044,  0.3171,  ..., -0.8625, -0.6403,  0.7204],
         [-0.5450, -1.6367,  0.2889,  ..., -0.8662, -0.6201,  0.6481]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.0360,  0.1090, -0.1890,  ...,  1.4989,  1.6122, -1.4643],
         [ 1.2186, -1.3370, -0.2337,  ...,  1.6266,  1.2361,  0.0886],
         ...,
         [ 0.6174, -2.0593, -0.4355,  ..., -1.2615, -0.1844, -0.3498],
         [ 0.6373, -2.13

Computing prompt embeddings:   2%|▏         | 50/2200 [00:01<01:09, 30.86it/s]


ERROR in extract_text_embeddings_batch: Could not extract 512-dim projected CLIP features from model output type: <class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>. Object: BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.0360,  0.1090, -0.1890,  ...,  1.4989,  1.6122, -1.4643],
         [ 1.2186, -1.3370, -0.2337,  ...,  1.6266,  1.2361,  0.0886],
         ...,
         [-0.3695, -0.9486,  0.7089,  ..., -0.9234, -0.6332, -0.5198],
         [-0.3746, -1.0124,  0.7639,  ..., -0.9738, -0.5678, -0.5414],
         [-0.3233, -1.0649,  0.7650,  ..., -0.9656, -0.5298, -0.5655]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.0360,  0.1090, -0.1890,  ...,  1.4989,  1.6122, -1.4643],
         [ 1.2186, -1.3370, -0.2337,  ...,  1.6266,  1.2361,  0.0886],
         ...,
         [-0.5916, -1.7089, -0.4374,  ..., -0.1074, -1.2703,  0.8683],
         [-0.5777, -1.73

...


ERROR in extract_text_embeddings_batch: Could not extract 512-dim projected CLIP features from model output type: <class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>. Object: BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.8184, -0.9818, -0.3273,  ...,  0.2208,  0.5668, -1.2833],
         [ 0.9884,  0.7219,  0.2865,  ..., -0.2374,  0.0594, -0.4277],
         ...,
         [ 1.4363, -2.0461, -0.1567,  ..., -1.5876,  0.4781,  0.1282],
         [ 1.4324, -2.0398, -0.1372,  ..., -1.4345,  0.5652,  0.1195],
         [ 1.3569, -1.9678, -0.1568,  ..., -1.4598,  0.3698,  0.3304]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 0.8184, -0.9818, -0.3273,  ...,  0.2208,  0.5668, -1.2833],
         [ 0.9884,  0.7219,  0.2865,  ..., -0.2374,  0.0594, -0.4277],
         ...,
         [ 0.2404, -1.0340, -0.7526,  ..., -2.0526,  0.5163, -0.2596],
         [ 0.2922, -1.07

Computing prompt embeddings:  99%|█████████▉| 2182/2200 [01:14<00:00, 31.90it/s]


ERROR in extract_text_embeddings_batch: Could not extract 512-dim projected CLIP features from model output type: <class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>. Object: BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.8301, -0.1343,  0.1393,  ...,  0.2781, -0.4305,  0.1571],
         [ 2.1219,  0.7889,  0.3013,  ..., -0.7244, -0.4014,  0.2146],
         ...,
         [ 1.8881, -0.8357, -0.4620,  ..., -2.0723,  0.1130, -0.4143],
         [ 1.9292, -0.8542, -0.4834,  ..., -2.1034,  0.1048, -0.4125],
         [ 1.8755, -0.9341, -0.3880,  ..., -2.2393,  0.1913, -0.3201]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.8301, -0.1343,  0.1393,  ...,  0.2781, -0.4305,  0.1571],
         [ 2.1219,  0.7889,  0.3013,  ..., -0.7244, -0.4014,  0.2146],
         ...,
         [ 1.9824, -0.8528,  0.0480,  ..., -1.4589, -0.1277, -0.1661],
         [ 2.0045, -0.89

Computing prompt embeddings: 100%|██████████| 2200/2200 [01:15<00:00, 29.25it/s]


ERROR in extract_text_embeddings_batch: Could not extract 512-dim projected CLIP features from model output type: <class 'transformers.modeling_outputs.BaseModelOutputWithPooling'>. Object: BaseModelOutputWithPooling(last_hidden_state=tensor([[[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.8301, -0.1343,  0.1393,  ...,  0.2781, -0.4305,  0.1571],
         [ 2.1219,  0.7889,  0.3013,  ..., -0.7244, -0.4014,  0.2146],
         ...,
         [ 1.9090, -0.9907, -0.0438,  ..., -1.9012, -0.4731, -0.3103],
         [ 1.9332, -1.0416, -0.0732,  ..., -1.8910, -0.4218, -0.2738],
         [ 1.9277, -1.1047, -0.0826,  ..., -1.9368, -0.4050, -0.2642]],

        [[ 0.3393,  0.1165,  0.1020,  ...,  0.2468,  0.5906,  0.1013],
         [ 1.8301, -0.1343,  0.1393,  ...,  0.2781, -0.4305,  0.1571],
         [ 2.1219,  0.7889,  0.3013,  ..., -0.7244, -0.4014,  0.2146],
         ...,
         [ 1.6869,  0.7039,  0.6831,  ..., -0.2355, -0.2354, -0.2847],
         [ 1.2161, -0.37

================================================================

STEP 7 IMAGE EMBEDDING EXTRACTION (WITH PREPROCESSING)


    Extract CLIP embedding with comprehensive preprocessing.
    
    Pipeline:
    1. Validate image
    2. Preprocess (EXIF, RGB conversion, aspect ratio)
    3. CLIP processor (normalization, tensor conversion)
    4. Extract embedding
    5. L2 normalization
    
    FIXED: Properly handles CLIP output tensors


================================================================

In [50]:
def extract_prompts_for_image(image_embedding, theme, prompt_embeddings, top_k=2):
    """Extract top prompts using theme-first approach."""

    if theme not in prompt_embeddings:
        return {}

    theme_prompts = prompt_embeddings[theme]
    extracted_prompts = {}

    for category, prompts in theme_prompts.items():
        scores = []

        for prompt_data in prompts:
            score = np.dot(image_embedding, prompt_data['embedding'])
            scores.append((prompt_data['text'], float(score)))

        top_prompts = sorted(scores, key=lambda x: x[1], reverse=True)[:top_k]
        filtered = [(text, score) for text, score in top_prompts if score > 0.40]

        if filtered:
            extracted_prompts[category] = [
                {'text': text, 'score': score} for text, score in filtered
            ]

    return extracted_prompts



def process_all_images(metadata, landmarks_path, model, processor, preprocessor, device):
    """Process all images with preprocessing."""

    image_embeddings = {}
    image_metadata_list = {}

    total_images = metadata['total_images']
    print(f"Processing {total_images} images...")

    stats = {'processed': 0, 'failed': 0}

    with tqdm(total=total_images, desc="Extracting embeddings") as pbar:
        for theme in metadata['themes']:
            theme_name = theme['theme_name']

            for state in theme['states']:
                state_name = state['state_name']

                for destination in state['destinations']:
                    dest_id = destination['destination_id']
                    dest_folder = destination['folder']

                    for img_filename in destination['images']:
                        image_id = f"{dest_id}_{img_filename.replace('.jpg', '').replace('.png', '')}"
                        image_path = os.path.join(landmarks_path, dest_folder, img_filename)

                        embedding = extract_image_embedding(image_path, model, processor, preprocessor, device)

                        if embedding is not None:
                            image_embeddings[image_id] = embedding

                            image_metadata_list[image_id] = {
                                'image_path': image_path,
                                'filename': img_filename,
                                'destination_id': dest_id,
                                'destination_name': destination['destination_name'],
                                'theme': theme_name,
                                'state': state_name,
                                'folder': dest_folder
                            }
                            stats['processed'] += 1
                        else:
                            stats['failed'] += 1

                        pbar.update(1)

    print(f"\nProcessed: {stats['processed']}")
    if stats['failed'] > 0:
        print(f"Failed: {stats['failed']}")

    return image_embeddings, image_metadata_list, stats


image_embeddings, image_metadata_dict, validation_stats = process_all_images(
    metadata, LANDMARKS_PATH, model, processor, preprocessor, device
)

print("="*80)


Processing 215 images...


Extracting embeddings: 100%|██████████| 215/215 [00:53<00:00,  4.02it/s]


Processed: 215


===========================================================================================================

STEP 8: Prompt Extraction For all images

===========================================================================================================

In [54]:
def process_all_prompts(image_embeddings, image_metadata_dict, prompt_embeddings):
    """Extract prompts for all images."""

    all_image_prompts = {}

    print(f"Extracting prompts for {len(image_embeddings)} images...")

    with tqdm(total=len(image_embeddings), desc="Extracting prompts") as pbar:
        for image_id, embedding in image_embeddings.items():
            metadata = image_metadata_dict[image_id]
            theme = metadata['theme']

            prompts = extract_prompts_for_image(embedding, theme, prompt_embeddings, top_k=2)

            all_image_prompts[image_id] = {
                'image_id': image_id,
                'theme': theme,
                'destination_id': metadata['destination_id'],
                'extracted_prompts': prompts,
                'total_prompts_extracted': sum(len(p) for p in prompts.values())
            }

            pbar.update(1)

    print(f"Extracted prompts for {len(all_image_prompts)} images")
    return all_image_prompts


all_image_prompts = process_all_prompts(image_embeddings, image_metadata_dict, prompt_embeddings)

print("="*80)


Extracting prompts for 215 images...


Extracting prompts: 100%|██████████| 215/215 [00:00<00:00, 51796.40it/s]

Extracted prompts for 215 images


===========================================================================================================

STEP 9: AGGREGATE PER DESTINATION

===========================================================================================================

In [56]:
print("\n" + "="*80)
print("STAGE 9: AGGREGATING BY DESTINATION")
print("="*80)

def aggregate_destination_embeddings(image_embeddings, image_metadata_dict):
    """Aggregate embeddings per destination."""

    destination_embeddings = {}
    destination_image_embeddings = {}

    for image_id, embedding in image_embeddings.items():
        dest_id = image_metadata_dict[image_id]['destination_id']

        if dest_id not in destination_image_embeddings:
            destination_image_embeddings[dest_id] = []

        destination_image_embeddings[dest_id].append(embedding)

    for dest_id, embeddings in destination_image_embeddings.items():
        embeddings_array = np.array(embeddings)
        avg_embedding = np.mean(embeddings_array, axis=0)
        avg_embedding = avg_embedding / np.linalg.norm(avg_embedding)

        destination_embeddings[dest_id] = {
            'average_embedding': avg_embedding,
            'individual_embeddings': embeddings_array,
            'num_images': len(embeddings)
        }

    print(f"Aggregated {len(destination_embeddings)} destinations")
    return destination_embeddings


def aggregate_destination_prompts(all_image_prompts):
    """Aggregate prompts with weighted scoring."""

    destination_prompts = {}
    dest_groups = {}

    for image_id, data in all_image_prompts.items():
        dest_id = data['destination_id']
        if dest_id not in dest_groups:
            dest_groups[dest_id] = []
        dest_groups[dest_id].append(data)

    for dest_id, images_data in dest_groups.items():
        num_images = len(images_data)
        category_prompts = {}

        for img_data in images_data:
            for category, prompts in img_data['extracted_prompts'].items():
                if category not in category_prompts:
                    category_prompts[category] = []
                for prompt in prompts:
                    category_prompts[category].append(prompt)

        aggregated = {}

        for category, prompts in category_prompts.items():
            prompt_stats = {}

            for prompt in prompts:
                text = prompt['text']
                score = prompt['score']

                if text not in prompt_stats:
                    prompt_stats[text] = {'scores': [], 'count': 0}

                prompt_stats[text]['scores'].append(score)
                prompt_stats[text]['count'] += 1

            weighted_prompts = []
            for text, stats in prompt_stats.items():
                avg_score = np.mean(stats['scores'])
                frequency = stats['count'] / num_images
                weighted_score = (avg_score * 0.6) + (frequency * 0.4)

                weighted_prompts.append({
                    'text': text,
                    'avg_score': float(avg_score),
                    'frequency': float(frequency),
                    'weighted_score': float(weighted_score),
                    'appearances': stats['count']
                })

            top_prompts = sorted(weighted_prompts, key=lambda x: x['weighted_score'], reverse=True)[:2]

            if top_prompts:
                aggregated[category] = top_prompts

        destination_prompts[dest_id] = {
            'destination_id': dest_id,
            'num_images': num_images,
            'aggregated_prompts': aggregated,
            'dominant_characteristics': {
                cat: prompts[0]['text'] for cat, prompts in aggregated.items() if prompts
            }
        }

    print(f"Aggregated prompts for {len(destination_prompts)} destinations")
    return destination_prompts


destination_embeddings = aggregate_destination_embeddings(image_embeddings, image_metadata_dict)
destination_prompts = aggregate_destination_prompts(all_image_prompts)

print("="*80)


STAGE 9: AGGREGATING BY DESTINATION
Aggregated 47 destinations
Aggregated prompts for 47 destinations


==========================================================================================================
STEP 10: SAVE ALL DATA

===========================================================================================================

In [57]:
print("\n" + "="*80)
print("STAGE 10: SAVING DATA")
print("="*80)

print("Saving embeddings...")

image_ids = list(image_embeddings.keys())
embeddings_array = np.array([image_embeddings[img_id] for img_id in image_ids])

destination_ids = list(destination_embeddings.keys())
dest_avg_embs = np.array([destination_embeddings[d_id]['average_embedding'] for d_id in destination_ids])

np.savez(
    f'{EMBEDDINGS_PATH}/all_embeddings.npz',
    image_ids=image_ids,
    image_embeddings=embeddings_array,
    destination_ids=destination_ids,
    destination_embeddings=dest_avg_embs
)

print(f"all_embeddings.npz ({embeddings_array.shape})")

embedding_index = {
    'image_index': {img_id: idx for idx, img_id in enumerate(image_ids)},
    'destination_index': {d_id: idx for idx, d_id in enumerate(destination_ids)},
    'metadata': {
        'created_date': datetime.now().isoformat(),
        'total_images': len(image_ids),
        'total_destinations': len(destination_ids),
        'embedding_dim': embeddings_array.shape[1],
        'model_name': model_name,
        'preprocessing_enabled': True,
        'validation_stats': validation_stats,
        'prompt_validation': validation_results
    }
}

with open(f'{EMBEDDINGS_PATH}/embedding_index.json', 'w') as f:
    json.dump(embedding_index, f, indent=2)

print(f"embedding_index.json")

with open(f'{PROMPTS_PATH}/image_prompts.json', 'w') as f:
    json.dump(all_image_prompts, f, indent=2)

print(f"image_prompts.json")

with open(f'{PROMPTS_PATH}/destination_prompts.json', 'w') as f:
    json.dump(destination_prompts, f, indent=2)

print(f"destination_prompts.json")

with open(f'{EMBEDDINGS_PATH}/destination_embeddings_detailed.pkl', 'wb') as f:
    pickle.dump(destination_embeddings, f)

print(f"destination_embeddings_detailed.pkl")

print("="*80)


STAGE 5: SAVING DATA
Saving embeddings...
all_embeddings.npz ((215, 768))
embedding_index.json
image_prompts.json
destination_prompts.json
destination_embeddings_detailed.pkl


==========================================================================================================
STEP 11: UPDATE METADATA

===========================================================================================================

In [61]:
print("\n" + "="*80)
print("STAGE 11: UPDATING METADATA")
print("="*80)

metadata['pipeline_status']['embeddings_computed'] = True
metadata['pipeline_status']['prompts_extracted'] = True
metadata['pipeline_status']['prompts_validated'] = True
metadata['last_updated'] = datetime.now().isoformat()
metadata['vl_encoding_version'] = '4.1'

for theme in metadata['themes']:
    for state in theme['states']:
        for destination in state['destinations']:
            dest_id = destination['destination_id']

            if dest_id in destination_embeddings:
                destination['embeddings_computed'] = True
                destination['prompts_extracted'] = True

                destination['embedding_references'] = {
                    'embeddings_file': 'vl_encoding/embeddings/all_embeddings.npz',
                    'destination_index': embedding_index['destination_index'][dest_id]
                }

                if dest_id in destination_prompts:
                    destination['dominant_prompts'] = destination_prompts[dest_id]['dominant_characteristics']

with open(METADATA_PATH, 'w') as f:
    json.dump(metadata, f, indent=2)

print(f"Updated metadata.json")
print("="*80)


STAGE 11: UPDATING METADATA
Updated metadata.json


========================================================================================================== STEP 12: FINAL SUMMARY

===========================================================================================================

In [59]:
print("\n" + "="*80)
print("SUCCESS! VL ENCODING COMPLETE")
print("="*80)

print(f"\nImages processed: {len(image_embeddings)}")
print(f"Destinations: {len(destination_embeddings)}")
print(f"Embedding dimension: {embeddings_array.shape[1]}")
print(f"Model: {model_name}")

print(f"\nPrompt Validation:")
print(f"  Total prompts: {validation_results['total_prompts']}")
print(f"  Valid prompts: {validation_results['valid_prompts']}")
print(f"  Issues found: {validation_results['prompts_with_issues']}")

print("\nOutput Files:")
print(f"  {EMBEDDINGS_PATH}/all_embeddings.npz")
print(f"  {EMBEDDINGS_PATH}/embedding_index.json")
print(f"  {EMBEDDINGS_PATH}/destination_embeddings_detailed.pkl")
print(f"  {PROMPTS_PATH}/image_prompts.json")
print(f"  {PROMPTS_PATH}/destination_prompts.json")
print(f"  {REPORTS_PATH}/prompt_validation_report.txt")

print("\nCategory-Based Prompts Available:")
categories = set()
for img_prompts in all_image_prompts.values():
    categories.update(img_prompts['extracted_prompts'].keys())
print(f"  Categories extracted: {', '.join(sorted(categories))}")

print("\nReady for Category-Aware Matching!")
print("="*80)


SUCCESS! VL ENCODING COMPLETE

Images processed: 215
Destinations: 47
Embedding dimension: 768
Model: openai/clip-vit-base-patch32

Prompt Validation:
  Total prompts: 2200
  Valid prompts: 2200
  Issues found: 0

Output Files:
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/embeddings/all_embeddings.npz
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/embeddings/embedding_index.json
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/embeddings/destination_embeddings_detailed.pkl
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/prompts/image_prompts.json
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/prompts/destination_prompts.json
  /content/drive/MyDrive/visual-intelligence-travel-finance/data/vl_encoding/reports/prompt_validation_report.txt

Category-Based Prompts Available:
  Categories extracted: 

Ready for Category-Aware Matching!
